In [5]:
import csv
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import webdriver_manager
from webdriver_manager.firefox import GeckoDriverManager

In [6]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

url = 'https://www.amazon.com'

driver.get(url)



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [C:\Users\Alexander\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache
C:\Users\Alexander\AppData\Local\Temp\ipykernel_25976\2399397005.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [7]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [8]:
url = get_url('linux book')
print(url)

https://www.amazon.com/s?k=linux+book&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1


In [9]:
driver.get(url)

Extract the Collection

In [10]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [11]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [12]:
len(results)

22

Prototype The Record

In [13]:
item = results[0]

In [14]:
atag = item.h2.a

In [15]:
description = atag.text.strip()

In [16]:
url = 'https://www.amazon.com' + atag.get('href')

In [17]:
price_parent = item.find('span', 'a-price')

In [18]:
price = price_parent.find('span', 'a-offscreen').text

In [19]:
rating = item.i.text

In [20]:
review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'})

Generalize the Pattern

In [21]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
        
    
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    
    #rating = ''
    #review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [22]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'text'

ERROR HANDLING

In [23]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [24]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [25]:
records[0]

('Linux for Beginners: A Practical and Comprehensive Guide to Learn Linux Operating System and Master Linux Command Line. Contains Self-Evaluation Tests to Verify Your Learning Level',
 '$16.99',
 '',
 '',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A07723931RMPWW0T90ZE5&url=%2FLinux-Beginners-Practical-Comprehensive-Self-Evaluation%2Fdp%2F1671228081%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2QRDRMQSAP0KZ%26keywords%3Dlinux%2Bbook%26qid%3D1653148399%26sprefix%3Dlinux%252Caps%252C812%26sr%3D8-1-spons%26psc%3D1&qualifier=1653148399&id=7099368670258168&widgetName=sp_atf')

In [26]:
for row in records:
    print(row[1])

$16.99
$44.35
$19.49
$28.29
$11.49
$31.99
$16.99
$44.35
$19.59
$34.24
$16.89
$0.00
$53.99
$16.89
$17.79
$14.99
$37.99
$39.06
$23.76
$14.99
$0.00
$0.99


Getting To Next Page

In [27]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page'
    
    return url

In [1]:
import csv
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import webdriver_manager
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import numpy as np

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=2QRDRMQSAP0KZ&sprefix=linux%2Caps%2C812&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url += '&page'
    
    return url

def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

    
def main(search_term):
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    url = 'https://www.amazon.com'
    driver.get(url)

    records = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
        
    #get data from results.csv
    df= pd.read_csv('./results.csv')

    #Drop unneeded column
    df.drop(['Rating','ReviewCount','Url'],axis=1, inplace=True)

    #Drop duplicate data
    df=df.drop_duplicates()

    #Convert prices from objects to floats
    df[df.columns[1:]] = df[df.columns[1:]].replace('[\$,]', '', regex=True).astype(float)

    #sort data based on the price- low to high
    df=df.sort_values('Price')

    df=df.head(50)
    
    return df

In [2]:
main('linux book')



====== WebDriver manager ======
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/chrisculley/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache
/var/folders/1f/m6gbl0qn4dg7x_j_z05_99yw0000gn/T/ipykernel_97207/2779321210.py:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


,Description,Price
21,Linux for Beginners: Why You're Not Using Linu...,0.00
9,Linux: 3 Books in 1: Linux for Beginners + Lin...,0.00
7,Linux Pocket Guide: Essential Commands,11.49
20,Linux for Absolute Beginners: An Introduction ...,12.99
4,Linux Bible,14.16
16,LFM: Linux Field Manual,14.99
1,Learn Linux Quickly: A Comprehensive Guide for...,16.89
0,Linux for Beginners: A Practical and Comprehen...,16.99
13,Ubuntu Linux Unleashed 2021 Edition,17.39
2,"The Linux Command Line, 2nd Edition: A Complet...",17.99
